# 1. subsampling

seqtk으로 50,000개 subsampling을 해본다. 사용법은 [seqtk 깃헙](https://github.com/lh3/seqtk)에서 확인할 수 있다.

## 1.1. seqk 설치

Conda 명령어로 설치 가능하며 이미 설치되어 있기 때문에 생략한다.

## 1.2. 원본 데이터이름 확인

In [1]:
!ls -a data

.		02-Man_2.fastq	05-Man_1.fastq	07-Man_2.fastq	10-Man_1.fastq
..		03-Man_1.fastq	05-Man_2.fastq	08-Man_1.fastq	10-Man_2.fastq
01-Man_1.fastq	03-Man_2.fastq	06-Man_1.fastq	08-Man_2.fastq
01-Man_2.fastq	04-Man_1.fastq	06-Man_2.fastq	09-Man_1.fastq
02-Man_1.fastq	04-Man_2.fastq	07-Man_1.fastq	09-Man_2.fastq


## 1.3. seqk 명령어 사용

아래 명령어로 subsampling한 파일을 만든다. 귀찮으니까 파일은 덮어 써버린다. seed number `-s100`는 동일한 샘플링을 위해서 동일한 숫자를 사용했다.

In [2]:
!seqtk sample -s100 data/01-Man_1.fastq 50000 > sub_data/01-Man_1.fastq
!seqtk sample -s100 data/01-Man_2.fastq 50000 > sub_data/01-Man_2.fastq
!seqtk sample -s100 data/02-Man_1.fastq 50000 > sub_data/02-Man_1.fastq
!seqtk sample -s100 data/02-Man_2.fastq 50000 > sub_data/02-Man_2.fastq
!seqtk sample -s100 data/03-Man_1.fastq 50000 > sub_data/03-Man_1.fastq
!seqtk sample -s100 data/03-Man_2.fastq 50000 > sub_data/03-Man_2.fastq
!seqtk sample -s100 data/04-Man_1.fastq 50000 > sub_data/04-Man_1.fastq
!seqtk sample -s100 data/04-Man_2.fastq 50000 > sub_data/04-Man_2.fastq
!seqtk sample -s100 data/05-Man_1.fastq 50000 > sub_data/05-Man_1.fastq
!seqtk sample -s100 data/05-Man_2.fastq 50000 > sub_data/05-Man_2.fastq
!seqtk sample -s100 data/06-Man_1.fastq 50000 > sub_data/06-Man_1.fastq
!seqtk sample -s100 data/06-Man_2.fastq 50000 > sub_data/06-Man_2.fastq
!seqtk sample -s100 data/07-Man_1.fastq 50000 > sub_data/07-Man_1.fastq
!seqtk sample -s100 data/07-Man_2.fastq 50000 > sub_data/07-Man_2.fastq
!seqtk sample -s100 data/08-Man_1.fastq 50000 > sub_data/08-Man_1.fastq
!seqtk sample -s100 data/08-Man_2.fastq 50000 > sub_data/08-Man_2.fastq
!seqtk sample -s100 data/09-Man_1.fastq 50000 > sub_data/09-Man_1.fastq
!seqtk sample -s100 data/09-Man_2.fastq 50000 > sub_data/09-Man_2.fastq
!seqtk sample -s100 data/10-Man_1.fastq 50000 > sub_data/10-Man_1.fastq
!seqtk sample -s100 data/10-Man_2.fastq 50000 > sub_data/10-Man_2.fastq

# 2. Demultiplex

`qiime2`를 사용하기 위해서는 데이터를 `.qza`형식으로 변환한다. 아래 명령어를 위해 먼저 사용자는 귀찮게도 `manifest`파일을 만들고 TSV
형식으로 *Sample ID*와 *file_path*를 **수동**으로 지정해줘야 한다.

이렇게하면 Demultiplex는 자동으로 되는 것 같다. 생성되는 `.qza`파일의 이름은 `sub_50k_demux.qza`으로 한다.



In [3]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path sub_manifest \
  --output-path sub_50k_demux.qza \
  --input-format PairedEndFastqManifestPhred33V2

Imported sub_manifest as PairedEndFastqManifestPhred33V2 to sub_50k_demux.qza


아래에 명령어를 사용하면 다음의 결과를 확인할 수 있다.

- 샘플당 얼마나 많은 서열이 있는지
- to get a summary of the distribution of sequence qualities at each position in your sequence data.


In [4]:
!qiime demux summarize \
  --i-data sub_50k_demux.qza \
  --o-visualization output/sub_demux.qzv

Saved Visualization to: output/sub_demux.qzv


In [5]:
from qiime2 import Visualization
Visualization.load('output/sub_demux.qzv')

<visualization: Visualization uuid: 9302b1e4-2c1b-4a79-9195-93d9e9cd6eb1>

# 3. Denoise/ Cluster

서열의 품질을 보고 사용할 길이를 선택해야 한다. 위의 **quality plot** 결과를 볼때 forward는 34-260bp, reverse는 7-220bp 정도가 괜찮아 보인다.

> 어떻게 결정해야하는지 좀 더 찾아봐야 한다.

그런데 foward와 reverse 서열의 최소값이 35bp라고 빨간 글씨로 나오는데 무슨소리인지 모르겠다. 그리고 서열 품질 분포에 대한 그림이 안나옴.

## 2.1. DADA2

`QIIME2`에는 `DADA2`랑 `Deblur` 방법으로 Denoising이 지원되는데, 신경쓸것 적은건 DADA2 사용한다. 도움말에 의하면 같은 실험세트에서 얻은 서열데이터는 DADA2 추천이고 서로다른 실험에서 얻은 서열데이터 처리할때는 Deblur추천이라 한다. 실행 목적은 다음 목록과 같다.

1. reducing sequence errors
2. dereplicating sequence

In [6]:
!qiime dada2 denoise-paired \
  --i-demultiplexed-seqs sub_50k_demux.qza \
  --p-n-threads 0 \
  --p-trim-left-f 35 \
  --p-trim-left-r 35 \
  --p-trunc-len-f 260 \
  --p-trunc-len-r 260 \
  --o-table output/sub_table.qza \
  --o-representative-sequences output/sub_rep_seqs.qza \
  --o-denoising-stats output/sub_denoising_stats.qza

Saved FeatureTable[Frequency] to: output/sub_table.qza
Saved FeatureData[Sequence] to: output/sub_rep_seqs.qza
Saved SampleData[DADA2Stats] to: output/sub_denoising_stats.qza


## 2.2. Clustering

OTU 만들때 사용하는 것이라고 한다. `q2-vsearch` 메소드로 구현되어 있다. 사용법을 몰라서 튜토리얼 읽어보고 나중에 한다.

### TODO: q2-vsearch 해보기


# 3. Feature table

feature table이 `QIIME2` 분석의 핵심이다. feature table을 통해 아래 목록의 분석들을 할 수 있다.

1. Taxonomic classification of sequences
2. Alpha and beta diversity analyses
3. Many diversity analyses rely on the phylogeneric similarity between individual features
4. Differential abundance measurements determine which features are significantly more/less abundant in groups

> The world is your oyster, let's dive in

## 3.1. feature table 확인

FeatureTable[Frequency] 에 대한 정보가 있다.


In [7]:
!qiime feature-table summarize \
  --i-table output/sub_table.qza \
  --o-visualization output/sub_table.qzv \
  --m-sample-metadata-file metadata.tsv

Saved Visualization to: output/sub_table.qzv


In [8]:
Visualization.load('output/sub_table.qzv')

<visualization: Visualization uuid: 7c2c5250-7b61-4a17-84c2-11a1a3809395>

위의 결과를 다음 정보를 얻을 수 있다.

```
Number of samples	10
Number of features	3,523
Total frequency	189,214
```

또한 앞으로 분석에 사용될 `Sampling Depth` 값을 정하게 되는데, 어떤 기준으로 정하는지 정확하게 모르겠다. 튜토리얼을 보면 sample수가 절반(50%)일때의 값을 사용하는 듯하다. 그래서 interactive sample detail에서 바를 움직여가면서 **18278**일때 Retained 91,390 (48.30%) features in 5 (50.00%) samples at the specifed sampling depth 인것을 사용했다.

## 3.2. Representative Sequence 확인

FeatureData[Sequence]에 해당하는 정보

In [9]:
!qiime feature-table tabulate-seqs \
  --i-data output/sub_rep_seqs.qza \
  --o-visualization output/sub_rep_seqs.qzv

Saved Visualization to: output/sub_rep_seqs.qzv


In [10]:
Visualization.load('output/sub_rep_seqs.qzv')

<visualization: Visualization uuid: fb14c5b7-abe1-4d0d-9a0b-2b87dadfbc21>

위 결과로 아래 정보를 확인 할 수 있다.


Sequence Count | Min Length	| Max Length | Mean Length | Range | Standard Deviation
---------------|------------|------------|-------------|-------|---------------
3523 | 225 | 418 | 384.86 | 193 | 13.76

## 3.3. Denoising에 대한 통계 값

아래 명령어를 통해 시각화 할 수 있다.

In [11]:
!qiime metadata tabulate \
  --m-input-file output/sub_denoising_stats.qza \
  --o-visualization output/sub_denoising_stats.qzv

Saved Visualization to: output/sub_denoising_stats.qzv


In [12]:
Visualization.load('output/sub_denoising_stats.qzv')

<visualization: Visualization uuid: d66efe0b-f3e2-4f28-9236-921168a0a775>

`non-chimeric` 이 많은 것이 좋은 것이겠지 라고 생각한다.


# 4. Analyses

## 4.1. Generating a phylogenetic tree for diversity analysis

QIIME 2 analysis allows the use of phylogenetic trees for diversity metrics such as Faith’s Phylogenetic Diversity and UniFrac distance as well as feature-based analyses in Gneiss. The tree provides an inherent structure to the data, allowing us to consider an evolutionary relationship between organisms.

QIIME 2 offers several ways to construct a phylogenetic tree. For this tutorial, we’re going to create a fragment insertion tree using the q2-fragment-insertion plugin. The authors of the fragment insertion plugin suggest that it can outperform traditional alignment based methods based on short Illumina reads by alignment against a reference tree built out of larger sequences. Our command, qiime fragment-insertion sepp will use the representative sequences (a FeatureData[Sequence] artifact) we generated during denoising to create a phylogenetic tree where the sequences have been inserted into the greengenes 13_8 99% identity reference tree backbone.

### 트리만드는데 사용하는 플러그인

- de novo method = mafft-fasttree
- fragment insertion method = sepp

Sepp는 자기들 방법이 더 좋다고 설명함. [자세한 내용](https://github.com/qiime2/q2-fragment-insertion). 명령어 말고 API로도 사용이 가능 

```python
from qiime2.plugins import fragment_insertion
```

### 4.1.1 Sepp로 트리만들기

In [14]:
!qiime fragment-insertion sepp \
  --i-representative-sequences output/sub_rep_seqs.qza \
  --o-tree output/sub_tree.qza \
  --o-placements output/sub_tree_placements.qza \
  --p-threads 8  # update to a higher number if you can

Saved Phylogeny[Rooted] to: output/sub_tree.qza
Saved Placements to: output/sub_tree_placements.qza


## 4.2. Core-metrics-phylogenetic 분석

`--p-sampling-depth` 값을 알아내기 위해서는 DADA2 feature table summary를 보아야한다? 

### 4.2.1. Alpha Rarefaction and Selecting a Rarefaction Depth

We’ll use qiime diversity alpha-rarefaction to subsample the ASV table at different depths (between --p-min-depth and --p-max-depth) and calculate the alpha diversity using one or more metrics (--p-metrics). When we checked the feature table, we found that the sample with the fewest sequences in the denoised table has (12707) features and that the sample with the most has (25055) features. We want to set a maximum depth close to the maximum number of sequences. We also know that if we look at a sequencing depth around (18278) sequences per sample, we’ll be looking at information from (5) samples. So, let’s set this as our maximum sequencing depth.

위의 설명을 읽고 max-depth를 설정한다.

In [15]:
!qiime diversity alpha-rarefaction \
  --i-table output/sub_table.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/sub_alpha_rarefaction_curves.qzv \
  --p-max-depth 18278

Saved Visualization to: output/sub_alpha_rarefaction_curves.qzv


In [16]:
Visualization.load('output/sub_alpha_rarefaction_curves.qzv')

<visualization: Visualization uuid: 7eb382ca-d9a1-471b-9bdc-64cdc3a51fc3>

위 그림 설명

The visualization file will display two plots. The upper plot will display the alpha diversity (observed OTUs or shannon) as a function of the sampling depth. This is used to determine whether the richness or evenness has saturated based on the sampling depth. The rarefaction curve should “level out” as you approach the maximum sampling depth. Failure to do so, especially with a diversity-only metric such as observed OTUs or Faith’s PD diversity, may indicate that the richness in the samples has not been fully saturated.

The second plot shows the number of samples in each metadata category group at each sampling depth. This is useful to determine the sampling depth where samples are lost, and whether this may be biased by metadata column group values. Remember that rarefaction is a two-step process and samples that do not meet the rarefaction depth are filtered out of the table. We can use the curves to look at the number of samples by different metadata columns.

If you’re still unsure of the rarefaction depth, you can also use the sample summary to look at which samples are lost by supplying sample metadata to the feature table summary.

## 위 결과에서 생각할 점

- Based on the current rarefaction curve and sample summary, what sequencing depth would you pick? Why?
- In this case, we can retain 10 samples with a rarefaction depth of 12000 sequences/sample.
- Based on the sequencing depth and distribution of samples, we’ll use 12000 sequences/sample for this analysis. This will let us keep 9 of 10 high quality samples (discarding the one sample with sequencing depth below 12000 sequences/sample).

## Beta Rarefaction: Beta diversity rarefaction

### UPGM cluster

Unweighted Pair Group Method

In [52]:
!qiime diversity beta-rarefaction\
  --i-table output/sub_table.qza \
  --i-phylogeny output/sub_tree.qza \
  --m-metadata-file metadata.tsv \
  --p-metric unweighted_unifrac \
  --p-clustering-method upgma \
  --o-visualization output/sub_beta_rarefaction_curves.qzv \
  --p-sampling-depth 12000

Saved Visualization to: output/sub_beta_rarefaction_curves.qzv


In [53]:
Visualization.load('output/sub_beta_rarefaction_curves.qzv')

<visualization: Visualization uuid: 76a43358-56ad-4160-bbba-b8c822f1aa75>

In [71]:
!qiime diversity beta-rarefaction\
  --i-table output/sub_table.qza \
  --i-phylogeny output/sub_tree.qza \
  --m-metadata-file metadata.tsv \
  --p-metric aitchison \
  --p-clustering-method upgma \
  --o-visualization output/sub_beta_rarefaction_curves.qzv \
  --p-sampling-depth 12000

Saved Visualization to: output/sub_beta_rarefaction_curves.qzv


In [72]:
Visualization.load('output/sub_beta_rarefaction_curves.qzv')

<visualization: Visualization uuid: b6124927-b89c-440d-92a3-7f881286444f>

### 4.2.2. Core-metrics-phylogenetic 실행

The qiime diversity core-metrics-phylogenetic method wraps several other methods, and it’s worthwhile to note that the steps can also be executed independently.

In [17]:
!qiime diversity core-metrics-phylogenetic \
  --i-table output/sub_table.qza \
  --i-phylogeny output/sub_tree.qza \
  --m-metadata-file metadata.tsv \
  --p-sampling-depth 12000 \
  --output-dir output/core-metrics-results

Saved FeatureTable[Frequency] to: output/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] % Properties('phylogenetic') to: output/core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: output/core-metrics-results/observed_otus_vector.qza
Saved SampleData[AlphaDiversity] to: output/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: output/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix % Properties('phylogenetic') to: output/core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix % Properties('phylogenetic') to: output/core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: output/core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: output/core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: output/core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: output/core-metrics-results/weig

## 4.3. Alpha diversity

Alpha diversity asks whether the distribution of features within a sample (or groups of samples) differs between different conditions.

### 4.3.1. Faith’s phylogenetic diversity

In [18]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity output/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/core-metrics-results/faiths_pd_statistics.qzv

Saved Visualization to: output/core-metrics-results/faiths_pd_statistics.qzv


In [19]:
Visualization.load('output/core-metrics-results/faiths_pd_statistics.qzv')

<visualization: Visualization uuid: 3d79db6d-8054-4aa4-b7ea-030275b2d05b>

### 4.3.2.  Pielou’s evenness

a measure of community evenness

In [30]:
!qiime diversity alpha-group-significance \
 --i-alpha-diversity output/core-metrics-results/evenness_vector.qza \
 --m-metadata-file metadata.tsv \
 --o-visualization output/core-metrics-results/evenness_statistics.qzv

Saved Visualization to: output/core-metrics-results/evenness_statistics.qzv


In [31]:
Visualization.load('output/core-metrics-results/evenness_statistics.qzv')

<visualization: Visualization uuid: 7aa01142-7b14-4146-a214-2751c428ef31>

### 4.3.3. observed_otus_vector



In [36]:
!qiime diversity alpha-group-significance \
 --i-alpha-diversity output/core-metrics-results/observed_otus_vector.qza \
 --m-metadata-file metadata.tsv \
 --o-visualization output/core-metrics-results/observed_otus_vector.qzv

Saved Visualization to: output/core-metrics-results/observed_otus_vector.qzv


In [37]:
Visualization.load('output/core-metrics-results/observed_otus_vector.qzv')

<visualization: Visualization uuid: c47251eb-689c-4b56-be8e-4e629437d106>

### 4.3.4. shannon_vector


In [38]:
!qiime diversity alpha-group-significance \
 --i-alpha-diversity output/core-metrics-results/shannon_vector.qza \
 --m-metadata-file metadata.tsv \
 --o-visualization output/core-metrics-results/shannon_vector.qzv

Saved Visualization to: output/core-metrics-results/shannon_vector.qzv


In [39]:
Visualization.load('output/core-metrics-results/shannon_vector.qzv')

<visualization: Visualization uuid: 70e668a9-8a72-44f6-90f0-2242597abec5>

### 4.3.5. Simpson evenness measure E

Diversity that account for the number of organisms and number of species

```
Simpson, E.H. (1949). “Measurement of Diversity”. Nature. (163): 688
```

명령어 사용법은 https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282 참고

In [46]:
!qiime diversity alpha \
  --i-table output/sub_table.qza \
  --p-metric simpson_e \
  --o-alpha-diversity output/simpson_e.qza

Saved SampleData[AlphaDiversity] to: output/simpson_e_vector.qza


In [48]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity output/simpson_e_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/simpson_e_vector.qzv

Saved Visualization to: output/simpson_e_vector.qzv


In [49]:
Visualization.load('output/simpson_e_vector.qzv')

<visualization: Visualization uuid: 57028f21-12d2-4d0b-bf78-3f30d8e5f57c>

## 4.2. Beta diversity

Next, we’ll compare the structure of the microbiome communities using beta diversity. Start by making a visual inspection of the principle coordinates plots (PCoA) plots that were generated by q2-emperor and core-metrics-results/weighted_unifrac_emperor.qzv.

### 4.2.1. unweighted unifrac

메타데이터에서 group 데이터를 가지고 비교

In [32]:
!qiime diversity beta-group-significance \
  --i-distance-matrix output/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column group \
  --o-visualization output/core-metrics-results/unweighted-unifrac-donor-significance.qzv

Saved Visualization to: output/core-metrics-results/unweighted-unifrac-donor-significance.qzv


In [33]:
Visualization.load('output/core-metrics-results/unweighted-unifrac-donor-significance.qzv')

<visualization: Visualization uuid: e54ec586-ac1b-4067-98cf-1f8109ceb3d0>

### 4.2.2. weighted unifrac

메타데이터에서 group 데이터를 가지고 비교

In [34]:
!qiime diversity beta-group-significance \
  --i-distance-matrix output/core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column group \
  --o-visualization output/core-metrics-results/weighted-unifrac-donor-significance.qzv

Saved Visualization to: output/core-metrics-results/weighted-unifrac-donor-significance.qzv


In [35]:
Visualization.load('output/core-metrics-results/weighted-unifrac-donor-significance.qzv')

<visualization: Visualization uuid: 178c7e77-1245-4fb1-af48-78585a95bed7>

alpha, beta diversity 분석외에도 `output/core-metrics-result`폴더에는 이미 다양한 분석이 되어 있다. 무슨의미인지 나중에 알아보자.

## 4.3.  unweighted PCOA


In [40]:
!qiime emperor plot \
  --i-pcoa output/core-metrics-results/unweighted_unifrac_pcoa_results.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/core-metrics-results/unweighted_unifrac_emperor.qzv \
#   --p-custom-axes days-since-experiment-start \

Saved Visualization to: output/core-metrics-results/unweighted_unifrac_emperor.qzv


In [41]:
Visualization.load('output/core-metrics-results/unweighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: b03eed8b-a878-459e-b0db-b42212ff3d28>

## 4.4.  weighted PCOA



In [43]:
!qiime emperor plot \
  --i-pcoa output/core-metrics-results/weighted_unifrac_pcoa_results.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/core-metrics-results/weighted_unifrac_emperor.qzv

Saved Visualization to: output/core-metrics-results/weighted_unifrac_emperor.qzv


In [44]:
Visualization.load('output/core-metrics-results/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 2083ee09-2aa6-4bdc-adcf-00e904f5a298>

# 5. Taxonomic analysis

For this analysis, we’ll use a pre-trained naive Bayes machine-learning classifier that was trained to differentiate taxa present in the 99% Greengenes 13_8 reference set trimmed to 250 bp of the V4 hypervariable region

In [65]:
!wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza"

--2019-09-18 09:41:14--  https://data.qiime2.org/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 52.35.38.247
Connecting to data.qiime2.org (data.qiime2.org)|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza [following]
--2019-09-18 09:41:15--  https://s3-us-west-2.amazonaws.com/qiime2-data/2019.7/common/gg-13-8-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.212.248
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.212.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28373760 (27M) [application/x-www-form-urlencoded]
Saving to: `gg-13-8-99-515-806-nb-classifier.qza'

100%[======================================>] 28,373,760  5.66M/s   in 5.3s    

2019-09-18 09:41:21 (5.10 MB/s) - `gg-13-

In [20]:
!qiime feature-classifier classify-sklearn \
  --i-reads output/sub_rep_seqs.qza  \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --o-classification output/sub_taxonomy.qza

Saved FeatureData[Taxonomy] to: output/sub_taxonomy.qza


In [21]:
!qiime metadata tabulate \
  --m-input-file output/sub_taxonomy.qza \
  --o-visualization output/sub_taxonomy.qzv

Saved Visualization to: output/sub_taxonomy.qzv


In [22]:
!qiime feature-table tabulate-seqs \
  --i-data output/sub_rep_seqs.qza \
  --o-visualization output/sub_rep_set.qzv

Saved Visualization to: output/sub_rep_set.qzv


## 5.1. bar plots 으로 시각화

In [24]:
!qiime feature-table filter-samples \
  --i-table output/sub_table.qza \
  --p-min-frequency 12000 \
  --o-filtered-table output/table_12k.qza

Saved FeatureTable[Frequency] to: output/table_12k.qza


In [25]:
!qiime taxa barplot \
  --i-table output/table_12k.qza \
  --i-taxonomy output/sub_taxonomy.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/taxa_barplot.qzv

Saved Visualization to: output/taxa_barplot.qzv


In [26]:
Visualization.load('output/taxa_barplot.qzv')

<visualization: Visualization uuid: ba486309-9a90-48ab-9524-529269989528>

## 5.2. 엽록체 제외하고 시각화

https://docs.qiime2.org/2019.7/tutorials/filtering/ 에서 데이터 필터링하는 방법이 있다.

In [27]:
!qiime taxa filter-table \
  --i-table output/table_12k.qza\
  --i-taxonomy output/sub_taxonomy.qza  \
  --p-exclude chloroplast \
  --o-filtered-table output/no_chloroplast.qza

Saved FeatureTable[Frequency] to: output/no_chloroplast.qza


In [28]:
!qiime taxa barplot \
  --i-table output/no_chloroplast.qza \
  --i-taxonomy output/sub_taxonomy.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization output/no_taxa_barplot.qzv

Saved Visualization to: output/no_taxa_barplot.qzv


In [29]:
Visualization.load('output/no_taxa_barplot.qzv')

<visualization: Visualization uuid: 6a09aa32-bbfc-441c-a196-33e7425da9d5>

# 결론

